<a href="https://colab.research.google.com/github/Theeyecode/Housing-Stress-Canada/blob/eda/descriptive_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis for Canada Housing Survery Data 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


url = "https://drive.google.com/uc?id=11Y8p_9-CYw0tpGPFu-jlgzxzWOOVS43F"

In [2]:


df = pd.read_csv(url)
df.head()

,PUMFID,EHA_10,EHA_25,FP_05,DWI_05A,DWI_05B,DWI_05C,DWI_05D,NEI_05A,NEI_05B,...,PSTIR_GR,PVISMIN,PWSA_D15,P2DCT_20,P2DCT_25,PATT_05,PATT_10,PATT_15A,PATT_15B,VERDATE
0,63501,4,2,1,2,2,2,2,4,4,...,1,9,999.6,996,6,1,1,6,6,11/08/2025
1,63502,3,2,2,2,2,2,2,4,4,...,1,2,999.6,996,6,2,1,6,6,11/08/2025
2,63503,3,2,1,2,2,2,2,4,4,...,1,2,999.6,996,6,2,1,6,6,11/08/2025
3,63504,3,2,1,2,1,2,2,3,4,...,1,1,999.6,996,6,2,1,6,6,11/08/2025
4,63505,4,2,1,2,2,2,2,4,4,...,1,2,999.6,2,1,1,2,6,3,11/08/2025


Shape of the Raw data


In [3]:
df.shape

(38657, 103)

In [4]:
# df.dtypes
df.dtypes.value_counts()


,count
int64,98
float64,4
object,1


In [5]:
# Identify non-numeric columns (typically dates or text fields)
df.select_dtypes(include="object").columns.tolist()

['VERDATE']

In [6]:
# Convert verification date to datetime for proper handling
df["VERDATE"] = pd.to_datetime(df["VERDATE"], errors="coerce")

In [7]:
# # Check missing values per column (after initial load)
df.isna().sum().sort_values(ascending=False)

,0
PUMFID,0
EHA_10,0
EHA_25,0
FP_05,0
DWI_05A,0
...,...
PATT_05,0
PATT_10,0
PATT_15A,0
PATT_15B,0


In [8]:
# Get basic descriptive stats for numeric columns (unweighted, structure check)
df.describe().T

,count,mean,min,25%,50%,75%,max,std
PUMFID,38657.0,82829.0,63501.0,73165.0,82829.0,92493.0,102157.0,11159.459015
EHA_10,38657.0,2.888119,1.0,2.0,3.0,4.0,9.0,1.076674
EHA_25,38657.0,1.953126,1.0,2.0,2.0,2.0,9.0,0.327064
FP_05,38657.0,1.028119,1.0,1.0,1.0,1.0,9.0,0.294491
DWI_05A,38657.0,1.96337,1.0,2.0,2.0,2.0,9.0,0.446082
...,...,...,...,...,...,...,...,...
PATT_05,38657.0,6.103733,0.0,1.0,1.0,2.0,99.0,21.497314
PATT_10,38657.0,1.87803,1.0,1.0,1.0,2.0,9.0,1.892823
PATT_15A,38657.0,5.800321,1.0,6.0,6.0,6.0,9.0,1.272822
PATT_15B,38657.0,5.399669,1.0,6.0,6.0,6.0,9.0,1.733045


In [9]:
# # Identify columns that contain obvious reserved codes (e.g., 9, 96, 99, 999, etc.)
reserved_codes = [9, 96, 99, 996, 999, 999.6, 999.9, 99999996, 99999999, 99999999999]

reserved_check = {
    col: df[col].isin(reserved_codes).any()
    for col in df.columns
    if df[col].dtype != "object"
}

[k for k, v in reserved_check.items() if v]


['EHA_10',
 'EHA_25',
 'FP_05',
 'DWI_05A',
 'DWI_05B',
 'DWI_05C',
 'DWI_05D',
 'NEI_05A',
 'NEI_05B',
 'NEI_05C',
 'NEI_05D',
 'NEI_05E',
 'NEI_05F',
 'NEI_05G',
 'NEI_05H',
 'NEI_05I',
 'WSA_05',
 'SDH_05',
 'CER_05',
 'CER_20',
 'LIS_10',
 'COS_10',
 'COS_15',
 'GH_05',
 'GH_10',
 'PMINOR',
 'PCER_10',
 'PCER_15',
 'PCHN',
 'PCOS_05',
 'PDCT_05',
 'P1DCT_20',
 'P1DCT_25',
 'PDV_SAH',
 'PDV_SUIT',
 'PDWLTYPE',
 'PDWS_10A',
 'PDWS_10B',
 'PDWS_10C',
 'PDWS_10D',
 'PDWS_10E',
 'PDWS_10F',
 'PDWS_10G',
 'PDWS_10H',
 'PDWS_10I',
 'PDWS_10J',
 'PEHA_05A',
 'PEHA_05B',
 'PEHA_05C',
 'PEMPL',
 'PFTHB5YR',
 'PHGEDUC',
 'PHHSIZE',
 'PHHTTINC',
 'PHTYPE',
 'PLIS_05',
 'PNSC_15',
 'POWN_20',
 'POWN_80',
 'PPAC_05',
 'PPAC_10',
 'PPAC_23',
 'PPAC_30',
 'PPAC_35',
 'PPAC_45A',
 'PPAC_45C',
 'PPAC_45D',
 'PPAC_45E',
 'PPAC_45F',
 'PPAC_45G',
 'PPAC_45H',
 'PPAC_45I',
 'PPAC_45J',
 'PPAC_45K',
 'PPAC_45L',
 'PPAC_45M',
 'PPAC_45N',
 'PPAC_45O',
 'PRSPIMST',
 'P1SCR_05',
 'PSCR_10',
 'PSCR_25',
 'P

---
## [Task 1 : Handle Reserved Codes as NA](https://emmanuelolajubu90.atlassian.net/browse/SCRUM-12)

* Identify outcome vars (PCHN, PSTIR_GR) → confirm universe + eligibility rules

* Apply logic to convert reserved codes to NA for all relevant variables, without performing any recoding at this stage.

---

In [10]:
# 1. PCHN (Core Housing Need)

print("PCHN (Original Counts):")
df['PCHN'].value_counts(dropna=False).sort_index()

PCHN (Original Counts):


,count
PCHN,
1,6164
2,30938
9,1555


In [11]:
# Create a clean version: Map 9 to NaN
df['PCHN_Clean'] = df['PCHN'].replace({9: np.nan})

In [12]:
# Print the cleaned data count

print(df['PCHN_Clean'].value_counts(dropna=False).sort_index())
print('_'*50)
print(f"Records Excluded (Not Stated): {df['PCHN_Clean'].isna().sum()}")

PCHN_Clean
1.0     6164
2.0    30938
NaN     1555
Name: count, dtype: int64
__________________________________________________
Records Excluded (Not Stated): 1555


In [13]:
# 2. PSTIR_GR (Shelter-cost-to-income ratio group)

print("Definition: 1 (<30%), 2 (30-50%), 3 (50-100%), 4 (>=100%) \n")
print("Reserved Codes: 5 = Not Applicable, 9 = Not Stated \n")

print("PSTIR_GR (Original Counts):")
df['PSTIR_GR'].value_counts(dropna=False).sort_index()

Definition: 1 (<30%), 2 (30-50%), 3 (50-100%), 4 (>=100%) 

Reserved Codes: 5 = Not Applicable, 9 = Not Stated 

PSTIR_GR (Original Counts):


,count
PSTIR_GR,
1,28650
2,6440
3,2012
4,549
5,429
9,577


In [14]:
# Map 5 and 9 to NaN

df['PSTIR_GR_Clean'] = df['PSTIR_GR'].replace({5: np.nan, 9: np.nan})

In [15]:
print("\nPSTIR_GR cleaned data counts:")
display(df['PSTIR_GR_Clean'].value_counts(dropna=False).sort_index())
print(f"\nRecords Excluded (N/A or Not Stated): {df['PSTIR_GR_Clean'].isna().sum()}")


PSTIR_GR cleaned data counts:


,count
PSTIR_GR_Clean,
1.0,28650
2.0,6440
3.0,2012
4.0,549
NaN,1006



Records Excluded (N/A or Not Stated): 1006


In [16]:
# 3. Valid Universe Check : How many households are valid for BOTH measures?

valid_both = df.dropna(subset=['PCHN_Clean', 'PSTIR_GR_Clean'])
print(f"Total rows in dataset: {len(df)}")
print(f"Rows valid for BOTH PCHN and PSTIR_GR: {len(valid_both)}")

Total rows in dataset: 38657
Rows valid for BOTH PCHN and PSTIR_GR: 37102


In [29]:
# 3. EHA_10 (Difficulty meeting financial needs)
# Definition: 1=Very difficult to 5=Very easy
# Reserved Code: 9 = Not Stated

print("AUDIT: EHA_10 (Difficulty Meeting Financial Needs)")
print("_"*50)

print("EHA_10 (Original Counts):")
display(df['EHA_10'].value_counts(dropna=False).sort_index())

# Create a clean version: Map 9 to NaN
df['EHA_10_Clean'] = df['EHA_10'].replace({9: np.nan})

print("\nEHA_10 (Cleaned Counts):")
display(df['EHA_10_Clean'].value_counts(dropna=False).sort_index())
print(f"Records Excluded (Not Stated): {df['EHA_10_Clean'].isna().sum()}")

AUDIT: EHA_10 (Difficulty Meeting Financial Needs)
__________________________________________________
EHA_10 (Original Counts):


,count
EHA_10,
1,3733
2,9669
3,15527
4,6990
5,2652
9,86



EHA_10 (Cleaned Counts):


,count
EHA_10_Clean,
1.0,3733
2.0,9669
3.0,15527
4.0,6990
5.0,2652
NaN,86


Records Excluded (Not Stated): 86


In [31]:
# Optional: Create a binary 'Stress' flag for easier analysis later
# Logic: If household answered 'Very difficult'(1) or 'Difficult'(2) -> Stress = 1, Else 0
df['Financial_Hardship_Flag'] = np.where(df['EHA_10_Clean'].isin([1, 2]), 1, 0)
# Note: We keep NaNs as 0 or handled separately depending on your choice later,
# but usually for flags, we need to be careful not to label 'Unknown' as 'Not Stressed'.
# Better approach for strictly clean data:
df.loc[df['EHA_10_Clean'].isna(), 'Financial_Hardship_Flag'] = np.nan

print("\nDerived Feature: Financial_Hardship_Flag (1=Diff/Very Diff)")
display(df['Financial_Hardship_Flag'].value_counts(dropna=False))


Derived Feature: Financial_Hardship_Flag (1=Diff/Very Diff)


,count
Financial_Hardship_Flag,
0.0,25169
1.0,13402
NaN,86


In [33]:
# 4. EHA_25 (Skipped or delayed mortgage/rent payment)
# Definition: 1 = Yes, 2 = No
# Reserved Code: 9 = Not Stated


print("AUDIT: EHA_25 (Skipped Payments)")

print("EHA_25 (Original Counts):")
display(df['EHA_25'].value_counts(dropna=False).sort_index())

# Create a clean version: Map 9 to NaN
df['EHA_25_Clean'] = df['EHA_25'].replace({9: np.nan})

print("\nEHA_25 (Cleaned Counts):")
display(df['EHA_25_Clean'].value_counts(dropna=False).sort_index())
print(f"Records Excluded (Not Stated): {df['EHA_25_Clean'].isna().sum()}")

AUDIT: EHA_25 (Skipped Payments)
EHA_25 (Original Counts):


,count
EHA_25,
1,2113
2,36501
9,43



EHA_25 (Cleaned Counts):


,count
EHA_25_Clean,
1.0,2113
2.0,36501
NaN,43


Records Excluded (Not Stated): 43


In [36]:
print("FINAL UNIVERSE CHECK")
print("_"*100)

# Check how many have valid data for ALL four outcome measures
valid_all = df.dropna(subset=['PCHN_Clean', 'PSTIR_GR_Clean', 'EHA_10_Clean', 'EHA_25_Clean'])

print(f"Total rows in dataset: {len(df)}")
print(f"Rows valid for ACADEMIC metrics (PCHN + PSTIR): {len(valid_both)}") # From your previous code
print(f"Rows valid for ALL metrics (Academic + Financial Stress): {len(valid_all)}")

# Check correlation between 'Academic' Need (PCHN) and 'Real' Stress (EHA_10)
# This is a quick sneak peek to see if the variables align
crosstab_check = pd.crosstab(df['PCHN_Clean'], df['EHA_10_Clean'], normalize='index')
print("\nQuick Validation: How does Core Housing Need relate to Financial Difficulty?")
print("_"*100)
display(crosstab_check)

FINAL UNIVERSE CHECK
____________________________________________________________________________________________________
Total rows in dataset: 38657
Rows valid for ACADEMIC metrics (PCHN + PSTIR): 37102
Rows valid for ALL metrics (Academic + Financial Stress): 36979

Quick Validation: How does Core Housing Need relate to Financial Difficulty?
____________________________________________________________________________________________________


EHA_10_Clean,1.0,2.0,3.0,4.0,5.0
PCHN_Clean,,,,,
1.0,0.184185,0.320371,0.321510,0.136349,0.037585
2.0,0.076736,0.235229,0.419927,0.192213,0.075894


### Output for Task 1

* **PCHN**: Preserved 37,102 valid households (6,164 In Need / 30,938 Not In Need) data. We excluded 1,555 "Not Stated" records.

* **PSTIR_GR**: Preserved 37,651 valid households. We excluded 1,006 records (429 "Not Applicable" + 577 "Not Stated") data.

* **Intersection**: 37,102 households have valid data for both variables, makes it valid sample data size for our analysis.

---
## [Task 2: Predictor Variable Audit](https://emmanuelolajubu90.atlassian.net/browse/SCRUM-13)

* **Goal**: "Sanitize" the independent variables (Demographics, Geography, Socio-economic).

* **Action**: Systematically identify reserved codes (e.g., 99, 99999996) for key columns like Income, Age, and Tenure to prevent them from skewing analysis.

---

In [37]:
# 1. Define Targeted Cleaning Rules (Based on CHS 2022 Data Dictionary)
# Key = Variable Name, Value = List of Reserved Codes to Convert to NaN
cleaning_rules = {
    'PDWLTYPE': [6, 99],    # 6=Valid Skip (CONFIRMED RESERVED), 99=Not Stated
    'PHTYPE':   [99],       # 06 is Valid (Two+ persons not in census family). Only clean 99.
    'PHGEDUC':  [99],       # 06 is Valid (Bachelor's degree). Only clean 99.
    'PDCT_05':  [6, 9],     # Tenure: 6=Valid Skip, 9=Not Stated
    'PMINOR':   [9],        # Presence of Minor: 9=Not Stated
    'PVISMIN':  [9],        # Visible Minority: 9=Not Stated (High missingness expected)
    'PEMPL':    [6, 9],     # Employment: 6=Valid Skip (<15/Inst.), 9=Not Stated
    'REGION':   [],         # No reserved codes (1-5 are valid regions)
    'PAGEP1':   []          # No reserved codes
}

In [38]:
# 2. Apply Cleaning Logic
print("PREDICTOR VARIABLE AUDIT LOG")
print(f"{'Variable':<15} | {'Status':<15} | {'Action Taken'}")
print("-" * 60)

for var, bad_codes in cleaning_rules.items():
    if not bad_codes:
        print(f"{var:<15} | {'Clean':<15} | Verified. No changes.")
    else:
        # Create a clean version of the column (e.g., PHGEDUC -> PHGEDUC_Clean)
        df[f'{var}_Clean'] = df[var].replace(bad_codes, np.nan)

        # Calculate impact
        n_removed = df[f'{var}_Clean'].isna().sum() - df[var].isna().sum()
        # Note: If original had NaNs, we subtract them. (PUMF usually has codes, not NaNs)
        # Simplified reporting:
        n_removed = df[var].isin(bad_codes).sum()

        print(f"{var:<15} | {'Sanitized':<15} | Replaced {bad_codes} with NaN ({n_removed} rows)")

PREDICTOR VARIABLE AUDIT LOG
Variable        | Status          | Action Taken
------------------------------------------------------------
PDWLTYPE        | Sanitized       | Replaced [6, 99] with NaN (13908 rows)
PHTYPE          | Sanitized       | Replaced [99] with NaN (2135 rows)
PHGEDUC         | Sanitized       | Replaced [99] with NaN (1279 rows)
PDCT_05         | Sanitized       | Replaced [6, 9] with NaN (539 rows)
PMINOR          | Sanitized       | Replaced [9] with NaN (2152 rows)
PVISMIN         | Sanitized       | Replaced [9] with NaN (11250 rows)
PEMPL           | Sanitized       | Replaced [6, 9] with NaN (1665 rows)
REGION          | Clean           | Verified. No changes.
PAGEP1          | Clean           | Verified. No changes.


In [43]:
# 3. Clean Continuous Variable: PHHTTINC (Income)
# Reserved Codes: 99999996 (Valid Skip), 99999999 (Not Stated), 99999999999 (Not Stated)
income_garbage = [99999996, 99999999, 99999999999]

df['PHHTTINC_Clean'] = df['PHHTTINC'].replace(income_garbage, np.nan)
inc_removed = df['PHHTTINC'].isin(income_garbage).sum()


print(f"PHHTTINC        | Sanitized       | Replaced reserved high-values ({inc_removed} rows)")
print('\n\n')
print(f"New Max Income: ${df['PHHTTINC_Clean'].max():,.0f}\n\n")

display(  df['PHHTTINC_Clean'].describe())

PHHTTINC        | Sanitized       | Replaced reserved high-values (2026 rows)



New Max Income: $975,000




,PHHTTINC_Clean
count,36631.000000
mean,84160.437198
std,82272.363137
min,-72500.000000
25%,30000.000000
50%,60000.000000
75%,110000.000000
max,975000.000000


In [20]:
# print("\nContinuous Variable: PHHTTINC")
# income_col = 'PHHTTINC'
# reserved_income_code = 99999999999
# max_val = df[income_col].max()

# if max_val == reserved_income_code:
#     count_reserved = (df[income_col] == reserved_income_code).sum()
#     print(f"-> Found reserved code {reserved_income_code} in {count_reserved} records.")

#     # Calculate stats on the clean data only
#     clean_income = df[df[income_col] != reserved_income_code][income_col]
#     print(f"-> Valid Income Range: ${clean_income.min():,.0f} to ${clean_income.max():,.0f}")
#     print(f"-> Median Income: ${clean_income.median():,.0f}")
# else:
#     print("-> No standard reserved code (99...9) found.")


Continuous Variable: PHHTTINC
-> Found reserved code 99999999999 in 2026 records.
-> Valid Income Range: $-72,500 to $975,000
-> Median Income: $60,000


# Data Retention Strategy

* **Goal**: Decide which categories to keep, drop, or collapse.

* **Action**: Review rare categories (e.g., small sample sizes in specific regions or gender groups) and decide if they need to be merged for statistical validity.

